In [3]:
%pip install yfinance
%pip install tqdm

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/14/03/3c854ca3e02eedf614abba4b2e177c469bf3af58207fa30d5098c5d652fe/yfinance-0.2.37-py2.py3-none-any.whl.metadata
  Using cached yfinance-0.2.37-py2.py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for pandas>=1.3.0 from https://files.pythonhosted.org/packages/71/00/6beaeeba7f075d15ea167a5caa039b861e58ff2f58a5b659abb9b544c8f6/pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata
  Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for numpy>=1.16.5 from https://files.pythonhosted.org/packages/16/2e/86f24451c2d530c88daf997cb8d6ac622c1d40d19f5a031ed68a4b73a374/numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Obtaining dependency information for requests>=2.31 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b4

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: c:\Users\Owner\.pyenv\pyenv-win\versions\3.12.1\python.exe -m pip install --upgrade pip


  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/2a/14/e75e52d521442e2fcc9f1df3c5e456aead034203d4797867980de558ab34/tqdm-4.66.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.6 kB 330.3 kB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.6 kB 375.8 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 378.8 kB/s eta 0:00:00
Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: c:\Users\Owner\.pyenv\pyenv-win\versions\3.12.1\python.exe -m pip install --upgrade pip


In [4]:
import yfinance as yf
import pandas as pd
import os
import json
from tqdm import tqdm

In [5]:
def fetch_sp500_daily_data():
    sp500_data = yf.download('^GSPC', start='2017-12-20', end='2018-06-10')
    return sp500_data

# Example usage
sp500_data = fetch_sp500_daily_data()
sp500_data = pd.DataFrame(sp500_data)

# Create a new dataframe to store the result
sp500_windows = pd.DataFrame()

# Iterate through each date in the dataframe
for date in sp500_data.index:
    close_values = []
    
    for i in range(-10, 11):
        offset_date = date + pd.DateOffset(days=i)
        if offset_date in sp500_data.index:
            close_value = sp500_data.loc[offset_date, 'Close']
        else:
            close_value = float('nan')
        close_values.append(close_value)
    
    result_row = {'Date': date}
    for i, close_value in zip(range(-10, 11), close_values):
        result_row[f'{i}'] = close_value

    sp500_windows = sp500_windows._append(result_row, ignore_index=True)

[*********************100%%**********************]  1 of 1 completed


In [15]:
# List of directories containing JSON files
directories = ['2018_01', '2018_02', '2018_03', '2018_04', '2018_05']  # Add all your directory paths here

# List to store extracted data
data_list = []

# Loop through directories
for directory in tqdm(directories):
    # List of JSON files in the directory
    json_files = [file for file in os.listdir(directory) if file.endswith('.json')]
        
    # Loop through JSON files in the directory
    for file_name in tqdm(json_files):
        # Read JSON data from file
        with open(os.path.join(directory, file_name), 'r', encoding='utf-8') as f:
            json_data = f.read()
        
        data = json.loads(json_data)

        data_list.append({
            'text': data['text'],
            'published': data['thread']['published'],
            'title': data['thread']['title']
        })

100%|██████████| 5/5 [21:30<00:00, 258.18s/it]


In [18]:
# Create DataFrame
news_articles = pd.DataFrame(data_list)
news_articles['published'] = pd.to_datetime(news_articles['published'], utc=True)
news_articles['published'] = news_articles['published'].dt.tz_convert(None)

# Merging the dataframes based on 'published' and 'date' columns
merged_df = pd.merge(news_articles, sp500_windows, left_on='published', right_on='Date', how='inner')

# Dropping the 'date' column as it's redundant
merged_df.drop(columns=['published'], inplace=True)

# Write to CSV
merged_df.to_csv('news_and_stocks.csv', index=False)

In [19]:
merged_df

,text,title,Date,-10,-9,-8,-7,-6,-5,-4,...,1,2,3,4,5,6,7,8,9,10
0,"Kate Samuelson, TIME Friday, January 05, 2018 ...",Winter storm forces world's largest passenger ...,2018-01-05,2680.50000,2682.620117,2687.540039,2673.610107,NaN,NaN,NaN,...,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.239990,NaN,NaN,NaN
1,"Phil Wahba, FORTUNE Photograph by Bloomberg ...",Barnes & Noble Shares Plunge 15% On Dismal Hol...,2018-01-05,2680.50000,2682.620117,2687.540039,2673.610107,NaN,NaN,NaN,...,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.239990,NaN,NaN,NaN
2,"Sy Mukherjee, FORTUNE Drew Angerer — Getty I...",The GOP May Attempt Major Health Care or Welfa...,2018-01-05,2680.50000,2682.620117,2687.540039,2673.610107,NaN,NaN,NaN,...,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.239990,NaN,NaN,NaN
3,"Chris Morris Friday, January 05, 2018 Credit: ...","Thanks, blizzard: Air travel may not get back ...",2018-01-05,2680.50000,2682.620117,2687.540039,2673.610107,NaN,NaN,NaN,...,NaN,NaN,2747.709961,2751.290039,2748.229980,2767.560059,2786.239990,NaN,NaN,NaN
4,"Bloomberg Tuesday, January 09, 2018 Credit: Su...",SpaceX's mysterious Zuma mission might not hav...,2018-01-09,NaN,NaN,NaN,2695.810059,2713.060059,2723.989990,2743.149902,...,2748.229980,2767.560059,2786.239990,NaN,NaN,NaN,2776.419922,2802.560059,2798.030029,2810.300049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3867,Home » Warren Buffett supplied to speculate $t...,Warren Buffett supplied to speculate $three bi...,2018-05-30,NaN,2733.010010,2724.439941,2733.290039,2727.760010,2721.330078,NaN,...,2705.270020,2734.620117,NaN,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN
3868,Italy's political turmoil could hit the contin...,"Italy crisis will be too much for Europe, Latv...",2018-05-31,2733.01001,2724.439941,2733.290039,2727.760010,2721.330078,NaN,NaN,...,2734.620117,NaN,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN,NaN
3869,Russia's Lavrov meets North Korea's Kim in Pyo...,Russia's Lavrov meets North Korea's Kim in Pyo...,2018-05-31,2733.01001,2724.439941,2733.290039,2727.760010,2721.330078,NaN,NaN,...,2734.620117,NaN,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN,NaN
3870,30 May\nCryptocurrency prices have been highly...,Bitcoin’s influence over cryptocurrency prices...,2018-05-30,NaN,2733.010010,2724.439941,2733.290039,2727.760010,2721.330078,NaN,...,2705.270020,2734.620117,NaN,NaN,2746.870117,2748.800049,2772.350098,2770.370117,2779.030029,NaN
